In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,SimpleRNN
import numpy as np


In [2]:
text="Hello!"
uniq=sorted(list(set(text)))

In [3]:
char_index={char:i for i,char in enumerate(uniq)}
index_char={i:char for i,char in enumerate(uniq)}

In [4]:
increment=1
max_len=3
sentance=[]
next_char=[]
for i in range(0,len(text)-max_len,increment):
  sentance.append(text[i:i+max_len])
  next_char.append(text[i+max_len])

In [5]:
print(sentance)
print(next_char)
print(uniq)

['Hel', 'ell', 'llo']
['l', 'o', '!']
['!', 'H', 'e', 'l', 'o']


In [6]:
X=np.zeros((len(sentance),max_len,len(uniq)),dtype=np.bool_)
y=np.zeros((len(sentance),len(uniq)),dtype=np.bool_)

In [7]:
for i,sentances in enumerate(sentance):
  for j,char in enumerate(sentances):
    X[i,j,char_index[char]]=1
  y[i,char_index[next_char[i]]]=1

In [19]:
print(X.shape)
print(y.shape)

(3, 3, 5)
(3, 5)


In [20]:
model=Sequential([
    LSTM(128,input_shape=(X.shape[1], X.shape[2])),
    Dense(64,activation='relu'),
    Dense(len(uniq),activation='softmax')
])

In [21]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 128)               68608     
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 5)                 325       
                                                                 
Total params: 77189 (301.52 KB)
Trainable params: 77189 (301.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
model.fit(X,y,batch_size=4,epochs=10,verbose=0)

In [25]:
seed_text="H"
generated_text=seed_text
for _ in range(len(text)):
  pred=np.zeros((1,max_len,len(uniq)),dtype=np.bool_)
  for ion,chars in enumerate(seed_text):
    pred[0,ion,char_index[chars]]=0.7
  prediction=model.predict(pred)[0]
  prediction=np.asarray(prediction).astype('float64')
  logits = np.log(prediction)
  temperature = 1.0
  scaled_logits = logits / temperature
  exp_logits = np.exp(scaled_logits)
  predicted = exp_logits / np.sum(exp_logits)
  probas = np.random.multinomial(1, predicted, 1)
  k=np.argmax(probas)
  generated_text+=index_char[k]

1/1 [==============================] - 0s 43ms/step


In [26]:
print(generated_text)

H!lHo!l
